In [1]:
import os

# Función para instalar OpenJDK 8 si no está instalado previamente.
def install_java():
    # Verifica si la ruta del JDK existe.
    if not os.path.exists('/usr/lib/jvm/java-8-openjdk-amd64'):
        print("Instalando OpenJDK 8...")
        # Comando para instalar Java en modo silencioso.
        !apt-get install openjdk-8-jdk-headless -qq > /dev/null
        print("OpenJDK 8 instalado correctamente.")
    else:
        print("OpenJDK 8 ya está instalado.")



# Función para descargar Apache Spark solo si no está ya descargado.
def download_spark():
    # URL y nombre del archivo comprimido de Spark.
    spark_url = "https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si el archivo comprimido ya existe.
    if not os.path.exists(spark_tar):
        print("Descargando Apache Spark...")
        # Comando para descargar el archivo desde la URL.
        !wget -q $spark_url
        print("Descarga completa.")
    else:
        print("El archivo de Apache Spark ya está descargado.")



# Función para descomprimir el archivo de Apache Spark solo si no está descomprimido.
def extract_spark():
    # Nombre de la carpeta descomprimida.
    spark_dir = "spark-3.4.3-bin-hadoop3"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si la carpeta descomprimida ya existe.
    if not os.path.exists(spark_dir):
        print("Descomprimiendo Apache Spark...")
        # Comando para descomprimir el archivo.
        !tar xf $spark_tar
        print("Apache Spark descomprimido.")
    else:
        print("La carpeta de Apache Spark ya existe.")



# Función para configurar las variables de entorno necesarias para Spark.
def set_environment_variables():
    # Establece la ruta de JAVA_HOME y SPARK_HOME.
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
    print("Variables de entorno configuradas.")



# Función para verificar si un paquete de Python ya está instalado.
import subprocess
import sys

def is_package_installed(package_name):
    try:
        # Ejecuta el comando `pip show` para verificar si el paquete está instalado.
        subprocess.check_call(
            [sys.executable, "-m", "pip", "show", package_name],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except subprocess.CalledProcessError:
        return False



# Función para instalar librerías de Python necesarias (findspark y pyspark).
def install_python_libraries():
    # Verifica e instala findspark.
    if not is_package_installed("findspark"):
        print("Instalando findspark...")
        !pip install -q findspark
    else:
        print("findspark ya está instalado.")

    # Verifica e instala pyspark.
    if not is_package_installed("pyspark"):
        print("Instalando pyspark...")
        !pip install -q pyspark
    else:
        print("pyspark ya está instalado.")



install_java()                # Instala OpenJDK 8
download_spark()              # Descarga Apache Spark
extract_spark()               # Descomprime Apache Spark
set_environment_variables()   # Configura las variables de entorno
install_python_libraries()    # Instala las librerías de Python

OpenJDK 8 ya está instalado.
El archivo de Apache Spark ya está descargado.
La carpeta de Apache Spark ya existe.
Variables de entorno configuradas.
findspark ya está instalado.
pyspark ya está instalado.


---

# **Crear una sesion en Spark**

In [2]:
# Importar la biblioteca findspark, que ayuda a configurar PySpark correctamente en el entorno de ejecución
import findspark

# Inicializar findspark para establecer las variables necesarias de PySpark
findspark.init()

# Importar SparkSession desde la biblioteca pyspark.sql
# SparkSession es la entrada principal para trabajar con DataFrames en PySpark
from pyspark.sql import SparkSession

# Crear o obtener una SparkSession
# Esto inicializa un entorno de Spark si no existe uno, o reutiliza uno existente
spark = SparkSession.builder.getOrCreate()

# Obtener el contexto de Spark (SparkContext) desde la SparkSession
# SparkContext es la entrada principal para trabajar con RDDs en PySpark
sc = spark.sparkContext


---

# **PARTICIONADO**

En Spark, una **partición** es la unidad más pequeña de datos que se puede procesar de forma paralela.

- Es un **fragmento de un conjunto de datos** (como un DataFrame o RDD) dividido en partes para distribuir el procesamiento entre los nodos del clúster.
- Cada partición se procesa de manera independiente, lo que permite a Spark manejar grandes volúmenes de datos de forma eficiente.

**Ejemplo:** Si tienes un archivo grande con 1 millón de filas y lo divides en 10 particiones, cada partición manejará 100,000 filas. Esto permite que Spark procese estas particiones en paralelo en diferentes nodos.


*Material de estudios:*

*   [Particionamiento de Datos en Apache Spark I](https://medium.com/iwannabedatadriven/particionamiento-de-datos-en-spark-df4eb0933e74)

---

#  **PARTICIONADORES EN SPARK**
**Diferencias entre HashPartitioner y RangePartitioner en Apache Spark**


Los particionadores en Spark determinan cómo se distribuyen los datos entre las particiones. Son clave para optimizar el procesamiento distribuido, especialmente en transformaciones como `groupByKey` o `reduceByKey`.

### 1. **HashPartitioner**
- **Cómo funciona:**
  - Divide los datos en particiones basándose en el **hash** de la clave.
  - El número de particiones se define explícitamente (por ejemplo, `numPartitions=4`).
  - Las claves con el mismo hash se colocan en la misma partición.

- **Uso común:**
  - Ideal para datos con claves distribuidas uniformemente.
  - Se usa automáticamente en transformaciones como `reduceByKey`.

- **Ejemplo:**
  Si tienes las claves `A`, `B`, `C` y `D`, el HashPartitioner calcula el hash de cada clave y las asigna a particiones, como:
  ```
  Partición 0: A, C
  Partición 1: B, D
  ```

---

### 2. **RangePartitioner**
- **Cómo funciona:**
  - Divide los datos en particiones en función de un **rango ordenado** de las claves.
  - Requiere que las claves tengan un orden definido (por ejemplo, valores numéricos o cadenas ordenables).
  - Se basa en muestreos de las claves para determinar los rangos de cada partición.

- **Uso común:**
  - Ideal para datos ordenados o cuando necesitas mantener un orden entre las particiones.
  - Se usa en aplicaciones que requieren búsquedas rápidas en rangos.

- **Ejemplo:**
  Si tienes claves numéricas `[1, 3, 5, 7, 9]` y 2 particiones:
  ```
  Partición 0: [1, 3, 5]
  Partición 1: [7, 9]
  ```

---

### **Diferencias clave:**
| Aspecto               | HashPartitioner                 | RangePartitioner                |
|-----------------------|---------------------------------|---------------------------------|
| **Distribución**      | Basada en el hash de la clave.  | Basada en rangos ordenados.     |
| **Orden de las claves** | No garantiza orden.            | Garantiza orden parcial.        |
| **Datos ideales**      | Claves uniformemente distribuidas. | Claves ordenadas o con rangos definidos. |

Ambos particionadores mejoran la eficiencia, pero elegir uno depende de los datos y la tarea.

# **Ejemplo práctico de HashPartitioner en Apache Spark: creación, distribución y análisis de particiones**

In [3]:
# Crear un RDD más grande con pares clave-valor
# Cada elemento del RDD es un par (clave, valor), donde la clave es un nombre de fruta y el valor es un número asociado.
rdd = sc.parallelize([("manzana", 10), ("naranja", 20), ("pera", 30),
                      ("uva", 40), ("mango", 50), ("piña", 60),
                      ("fresa", 70), ("kiwi", 80), ("cereza", 90),
                      ("sandía", 100), ("melon", 110), ("limón", 120)])

# Número de particiones
# Este valor define en cuántas partes queremos dividir los datos del RDD.
num_particiones = 3

# Paso 1: Aplicar HashPartitioner a nuestro RDD de clave-valor
# `partitionBy` utiliza un particionador basado en hash para asignar cada clave a una partición.
# Las claves con el mismo hash (módulo del número de particiones) irán a la misma partición.
partitioned_rdd = rdd.partitionBy(num_particiones)

# Paso 2: Verificar la distribución de los datos en las particiones
# Aquí usamos `mapPartitionsWithIndex` para inspeccionar cómo los datos están distribuidos en cada partición.
# La función lambda recibe el índice de la partición (idx) y los datos dentro de esa partición (it).
partitioned_data = partitioned_rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(it))],  # Convertimos los datos a una lista junto con el índice de partición.
    preservesPartitioning=True         # Indicamos que mantenemos la partición original del RDD.
)

# Mostrar los datos repartidos por partición
# Recorremos los resultados para imprimir qué datos se encuentran en cada partición.
print("Distribución de datos por partición:")
for idx, items in partitioned_data.collect():  # `collect` reúne todos los datos distribuidos para mostrarlos.
    print(f"Partición {idx}: {items}")

# Paso 3: Verificar el hash de cada clave y la partición asignada
# Para cada clave en el RDD original, calculamos su hash y determinamos en qué partición será asignada.
# La partición se calcula como: hash(clave) % num_particiones.
print("\nHash y partición asignada para cada clave:")
hashes = rdd.map(lambda x: (x[0], hash(x[0]), hash(x[0]) % num_particiones))

# Imprimimos los valores de hash y la partición calculada para cada clave
for clave, h, partition in hashes.collect():  # `collect` reúne los valores hash calculados para imprimirlos.
    print(f"Clave: {clave}, Hash: {h}, Partición: {partition}")


Distribución de datos por partición:
Partición 0: [('manzana', 10), ('naranja', 20), ('uva', 40), ('mango', 50), ('cereza', 90)]
Partición 1: [('pera', 30), ('fresa', 70), ('kiwi', 80)]
Partición 2: [('piña', 60), ('sandía', 100), ('melon', 110), ('limón', 120)]

Hash y partición asignada para cada clave:
Clave: manzana, Hash: -6212662399204218333, Partición: 0
Clave: naranja, Hash: 1245446572899628356, Partición: 0
Clave: pera, Hash: -3592559662256592542, Partición: 1
Clave: uva, Hash: 1289978012078186706, Partición: 0
Clave: mango, Hash: 7509992797015814037, Partición: 0
Clave: piña, Hash: -5831872772280901141, Partición: 2
Clave: fresa, Hash: 2182514332187640556, Partición: 1
Clave: kiwi, Hash: -8292140047809600230, Partición: 1
Clave: cereza, Hash: 614178665651577378, Partición: 0
Clave: sandía, Hash: -5422801810092505891, Partición: 2
Clave: melon, Hash: 5812577297748102197, Partición: 2
Clave: limón, Hash: 3012300008299366733, Partición: 2


---

# **Particionamiento de Datos en Spark con RangePartitioner: Implementación y Validación**

In [5]:
# Crear un RDD más grande con pares clave-valor
# Cada par contiene una clave (fruta) y un valor (número asociado).
rdd = sc.parallelize([
    ("manzana", 10), ("naranja", 20), ("pera", 30),
    ("uva", 40), ("mango", 50), ("piña", 60),
    ("fresa", 70), ("kiwi", 80), ("cereza", 90),
    ("sandía", 100), ("melon", 110), ("limón", 120)
])

# Paso 1: Determinar los rangos manualmente
# 1. Ordenamos las claves del RDD en orden alfabético.
sorted_keys = sorted(rdd.map(lambda x: x[0]).collect())  # Se extraen las claves y se ordenan.

# 2. Dividimos las claves ordenadas en rangos según el número de particiones.
num_particiones = 3  # Especificamos que queremos dividir los datos en 3 particiones.
# Calculamos los puntos de corte para los rangos en función del número de particiones.
rangos = [sorted_keys[i * len(sorted_keys) // num_particiones] for i in range(1, num_particiones)]
# `rangos` contiene los límites superiores de cada partición excepto la última.
print(f"Rangos para las particiones: {rangos}")  # Mostramos los rangos calculados.

# Paso 2: Función de partición personalizada
def range_partitioner(clave):
    """
    Función que asigna una clave a una partición basándose en los rangos predefinidos.
    Parámetros:
        clave (str): La clave a particionar (nombre de la fruta).
    Retorna:
        int: El índice de la partición correspondiente.
    """
    for i, limite in enumerate(rangos):  # Iteramos por los límites de los rangos.
        if clave < limite:  # Si la clave es menor que un límite, asignamos esa partición.
            return i
    return len(rangos)  # Si no es menor que ningún límite, se asigna a la última partición.

# Paso 3: Aplicar la partición al RDD
# Usamos `partitionBy` con el particionador personalizado para dividir los datos según los rangos.
partitioned_rdd = rdd.partitionBy(num_particiones, lambda k: range_partitioner(k))

# Paso 4: Inspeccionar las particiones
# Usamos `mapPartitionsWithIndex` para inspeccionar los datos de cada partición.
# Para cada partición, generamos una lista que incluye el índice de la partición y los datos de esta.
partitioned_data = partitioned_rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(it))],  # `idx` es el índice de la partición, `it` son los datos en ella.
    preservesPartitioning=True  # Indicamos que mantenemos la partición original.
)

# Mostramos los datos distribuidos en cada partición.
print("\nDistribución de datos por partición (simulación de RangePartitioner):")
for idx, items in partitioned_data.collect():  # Recolectamos los datos de las particiones y los imprimimos.
    print(f"Partición {idx}: {items}")

# Paso 5: Validar cómo las claves fueron organizadas por rango
print("\nValidación del orden de las claves dentro de cada partición:")
for idx, items in partitioned_data.collect():
    # Extraemos solo las claves de los datos en cada partición.
    claves = [item[0] for item in items]
    # Mostramos las claves en cada partición para verificar su organización.
    print(f"Partición {idx}: Claves ordenadas: {claves}")


Rangos para las particiones: ['mango', 'pera']

Distribución de datos por partición (simulación de RangePartitioner):
Partición 0: [('fresa', 70), ('kiwi', 80), ('cereza', 90), ('limón', 120)]
Partición 1: [('manzana', 10), ('naranja', 20), ('mango', 50), ('melon', 110)]
Partición 2: [('pera', 30), ('uva', 40), ('piña', 60), ('sandía', 100)]

Validación del orden de las claves dentro de cada partición:
Partición 0: Claves ordenadas: ['fresa', 'kiwi', 'cereza', 'limón']
Partición 1: Claves ordenadas: ['manzana', 'naranja', 'mango', 'melon']
Partición 2: Claves ordenadas: ['pera', 'uva', 'piña', 'sandía']


### Explicación de los resultados

**1. Rangos para las particiones: `['mango', 'pera']`**
- Los rangos definidos (`'mango'` y `'pera'`) dividen las claves en tres segmentos:
  - **Partición 0:** Claves menores a `'mango'`.
  - **Partición 1:** Claves mayores o iguales a `'mango'` pero menores a `'pera'`.
  - **Partición 2:** Claves mayores o iguales a `'pera'`.

---

**2. Distribución de datos por partición**

- **Partición 0:**
  Contiene las claves que son menores a `'mango'` alfabéticamente:
  - `'fresa'`, `'kiwi'`, `'cereza'`, y `'limón'`.

- **Partición 1:**
  Contiene las claves que están en el rango de `'mango'` a `'pera'` (excluyendo `'pera'`):
  - `'manzana'`, `'naranja'`, `'mango'`, y `'melon'`.

- **Partición 2:**
  Contiene las claves mayores o iguales a `'pera'`:
  - `'pera'`, `'uva'`, `'piña'`, y `'sandía'`.

---

**3. Validación del orden de las claves dentro de cada partición**
- Las claves en cada partición están ordenadas de acuerdo a su orden de aparición en el RDD original. Esto se debe a que **`partitionBy` no reordena los datos dentro de cada partición**, sino que simplemente los asigna a las particiones según la lógica del particionador.

---

**4. Por qué las claves se distribuyen así**
- El particionador utiliza las reglas de comparación de cadenas para determinar la posición alfabética de las claves. Las claves se comparan carácter por carácter siguiendo el estándar Unicode:
  - `'a'` < `'b'` < ... < `'z'`.
  - Por ejemplo, `'fresa'` es menor que `'mango'` porque `'f'` viene antes de `'m'`.

Dentro de cada partición, el orden de las claves refleja el orden en que aparecen originalmente en el RDD. Como no se aplicó una operación explícita para ordenar las claves dentro de las particiones, permanecen en su orden original.

---

### Resumen
- Los rangos definidos (`['mango', 'pera']`) dividen las claves en tres grupos basados en su posición alfabética.
- La distribución por particiones depende únicamente de estos rangos, y no hay un reordenamiento interno dentro de las particiones.
- Si se necesita que las claves dentro de cada partición estén ordenadas alfabéticamente, sería necesario aplicar una transformación adicional como `sortByKey`.

---
---

# **MEZCLA DE DATOS (shuffling)**

**Mezcla de Datos (Shuffling) en Spark**

La **mezcla de datos** (o *shuffling*) es un proceso crucial en sistemas distribuidos como Apache Spark, donde los datos se reorganizan y se redistribuyen a través de varias particiones o nodos del clúster. Este proceso suele ocurrir durante operaciones que implican una reordenación significativa de los datos, como `groupByKey()`, `reduceByKey()`, `join()`, `distinct()`, entre otras.

### ¿Por qué ocurre el shuffling?

En Spark, los datos están distribuidos en diferentes particiones dentro de un clúster. Sin embargo, muchas veces es necesario que los datos relacionados (por ejemplo, las claves de un `groupByKey`) estén en la misma partición para poder realizar el procesamiento adecuado. El *shuffling* es la forma en que Spark redistribuye los datos entre las particiones para que la operación pueda realizarse correctamente.

### Proceso de Shuffling

El proceso de *shuffling* implica tres pasos principales:

1. **Lectura de Datos**: Cada nodo o partición lee los datos que le corresponden y prepara el conjunto de datos que deben enviarse a otros nodos.
2. **Reorganización**: Los datos se envían a las particiones correspondientes según la clave o el criterio de la operación. En este paso, las claves que deben ser procesadas juntas (por ejemplo, las mismas claves en una operación de `groupByKey`) son movidas a las mismas particiones.
3. **Reescritura en las Particiones Destino**: Finalmente, los datos reorganizados se escriben de nuevo en las particiones, lo que permite que la operación final se realice en los datos correctos.

### Impacto en el Rendimiento

El *shuffling* es una operación costosa en términos de tiempo y recursos. Esto se debe a varias razones:

- **Transferencia de Datos**: Implica el movimiento de grandes cantidades de datos entre nodos, lo que puede ser lento dependiendo de la infraestructura de red.
- **Almacenamiento Intermedio**: Los datos deben almacenarse temporalmente en el disco antes de ser procesados, lo que puede generar latencia si el disco es lento.
- **Ordenación y Cálculos Adicionales**: El proceso de *shuffling* también implica ordenar los datos o aplicar otras transformaciones, lo cual aumenta la complejidad computacional.

### Minimizar el Shuffling

Aunque el *shuffling* es inevitable en algunas operaciones, existen técnicas para minimizar su impacto:

- **Usar operaciones más eficientes**: Por ejemplo, `reduceByKey()` en lugar de `groupByKey()`, ya que reduce los datos antes de que se distribuyan entre particiones.
- **Ajustar el número de particiones**: Controlar el número de particiones para evitar un número excesivo de movimientos de datos.
- **Evitar operaciones innecesarias**: Reducir el uso de operaciones que desencadenan *shuffling* a menos que sea absolutamente necesario.

En resumen, el *shuffling* es un proceso fundamental en Spark para distribuir y reorganizar los datos en función de las operaciones, pero es una de las operaciones más costosas en términos de rendimiento. Por lo tanto, es importante entender cómo y cuándo ocurre para optimizar el rendimiento de las aplicaciones distribuidas.

### Ejemplo 1: `groupByKey()`

En este ejemplo, vamos a usar `groupByKey()`, que es una operación que dispara un *shuffling* para agrupar los valores por su clave.

In [6]:
# Crear un RDD con pares clave-valor
rdd = sc.parallelize([("manzana", 10), ("naranja", 20), ("manzana", 15), ("uva", 30), ("naranja", 25)])

# Realizar un groupByKey() que agrupa los valores por clave
grouped_rdd = rdd.groupByKey()

# Ver los resultados
print("Resultado de groupByKey:")
for key, values in grouped_rdd.collect():
    print(f"Clave: {key}, Valores: {list(values)}")

Resultado de groupByKey:
Clave: naranja, Valores: [20, 25]
Clave: uva, Valores: [30]
Clave: manzana, Valores: [10, 15]


### Ejemplo 2: `reduceByKey()`

`reduceByKey()` es una operación eficiente que realiza un *shuffling* pero minimizando la cantidad de datos que se tienen que mover entre particiones, ya que reduce los valores por clave antes de realizar la mezcla de datos.

In [7]:
# Crear un RDD con pares clave-valor
rdd = sc.parallelize([("manzana", 10), ("naranja", 20), ("manzana", 15), ("uva", 30), ("naranja", 25)])

# Realizar un reduceByKey() para sumar los valores por clave
reduced_rdd = rdd.reduceByKey(lambda x, y: x + y)

# Ver los resultados
print("Resultado de reduceByKey:")
for key, value in reduced_rdd.collect():
    print(f"Clave: {key}, Suma: {value}")

Resultado de reduceByKey:
Clave: naranja, Suma: 45
Clave: uva, Suma: 30
Clave: manzana, Suma: 25


### Ejemplo 3: `join()`

El uso de `join()` entre dos RDDs también genera un *shuffling*, ya que los datos con claves coincidentes de ambos RDDs deben ser redistribuidos y reorganizados en las particiones.

In [8]:
# Crear dos RDDs con pares clave-valor
rdd1 = sc.parallelize([("manzana", 10), ("naranja", 20), ("uva", 30)])
rdd2 = sc.parallelize([("manzana", 5), ("naranja", 15), ("uva", 25)])

# Realizar un join entre los dos RDDs
joined_rdd = rdd1.join(rdd2)

# Ver los resultados
print("Resultado de join:")
for key, value in joined_rdd.collect():
    print(f"Clave: {key}, Valores: {value}")

Resultado de join:
Clave: naranja, Valores: (20, 15)
Clave: uva, Valores: (30, 25)
Clave: manzana, Valores: (10, 5)


### Ejemplo 4: `distinct()`

La operación `distinct()` realiza un *shuffling* para eliminar los elementos duplicados en un RDD.

In [12]:
# Crear un RDD con elementos duplicados
rdd = sc.parallelize([1, 2, 3, 4, 4, 5, 5, 6, 7, 8, 8])

# Realizar un distinct() para eliminar duplicados
distinct_rdd = rdd.distinct()

# Ver los resultados
print("Resultado de distinct:")
for value in distinct_rdd.collect():
    print(value)

Resultado de distinct:
2
4
6
8
1
3
5
7


### Ejemplo 5: `sortBy()`

La operación `sortBy()` también puede involucrar *shuffling* si los datos no están localmente ordenados en las particiones y Spark necesita redistribuir los datos para ordenar las claves.

In [10]:
# Crear un RDD con datos desordenados
rdd = sc.parallelize([("manzana", 30), ("uva", 10), ("naranja", 20), ("pera", 25)])

# Realizar un sortBy() para ordenar por el valor
sorted_rdd = rdd.sortBy(lambda x: x[1])

# Ver los resultados
print("Resultado de sortBy:")
for key, value in sorted_rdd.collect():
    print(f"Clave: {key}, Valor: {value}")

Resultado de sortBy:
Clave: uva, Valor: 10
Clave: naranja, Valor: 20
Clave: pera, Valor: 25
Clave: manzana, Valor: 30


### Ejemplo 5: `distinct() + sortBy()`

Si necesitas que los datos estén ordenados después de eliminar los duplicados, puedes aplicar una operación adicional de ordenación. Aquí te muestro cómo hacerlo:

In [13]:
# Crear un RDD con elementos duplicados
rdd = sc.parallelize([1, 2, 3, 4, 4, 5, 5, 6, 7, 8, 8])

# Realizar un distinct() para eliminar duplicados
distinct_rdd = rdd.distinct()

# Ordenar los resultados
sorted_rdd = distinct_rdd.sortBy(lambda x: x)

# Ver los resultados
print("Resultado de distinct y sortBy:")
for value in sorted_rdd.collect():
    print(value)

Resultado de distinct y sortBy:
1
2
3
4
5
6
7
8


### Conclusión

Estas operaciones son solo algunos ejemplos de cómo se puede realizar un *shuffling* en Spark. Cada vez que Spark necesita redistribuir los datos entre diferentes particiones, se activa el proceso de *shuffling*, lo que puede afectar el rendimiento si no se maneja adecuadamente.